In [1]:
#import modules
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import csv
from pandas.api.types import CategoricalDtype
low_memory = False

In [2]:
#read csv data from the CDC for CA
csv_ca_cdc_path = "./Resources/CA_cases_by_county/cdc_ca_output_data.csv"
ca_covid = pd.read_csv(csv_ca_cdc_path, encoding="utf-8")
ca_covid = pd.DataFrame(ca_covid)
ca_covid = ca_covid.drop(columns=['res_state'])
ca_covid.head()

C:\Users\Dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,case_month,res_county,age_group,current_status,sex,race,ethnicity
0,2020-12,SONOMA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino
1,2020-12,SACRAMENTO,18 to 49 years,Laboratory-confirmed case,Male,American Indian/Alaska Native,Hispanic/Latino
2,2021-01,SANTA CLARA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino
3,2020-11,SANTA CRUZ,18 to 49 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino
4,2020-11,TEHAMA,18 to 49 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino


In [3]:
#read in Population Data from the Department of Finance (which the CDC uses)
#as well as partial/full vacciantion numbers
ca_pop_vac_path = './Resources/CA_county_population_and_vaccination_data.csv'
ca_pop_vac = pd.read_csv(ca_pop_vac_path)
ca_popvac_df = pd.DataFrame(ca_pop_vac)

In [4]:
#make a list which counties are not included, though this may not be necessary for the final notebook.
#it is important to mention in the presentation, however.
print(len(ca_covid['res_county'].unique()))

cdc_included_counties = ca_covid['res_county'].unique()
cdc_included_counties = cdc_included_counties.tolist()

counties_list = ['Alameda', 'Alpine', 'Amador' ,'Butte' ,'Calaveras' ,'Colusa' ,'Contra Costa' ,'Del Norte'
                 ,'El Dorado' ,'Fresno' ,'Glenn' ,'Humboldt' ,'Imperial' ,'Inyo' ,'Kern' ,'Kings' ,'Lake' ,'Lassen',
                 'Los Angeles' ,'Madera' ,'Marin' ,'Mariposa' ,'Mendocino' ,'Merced' ,'Modoc' ,'Mono' ,'Monterey' ,'Napa', 
                 'Nevada' ,'Orange' ,'Placer' ,'Plumas' ,'Riverside' ,'Sacramento' ,'San Benito' ,'San Bernardino' ,'San Diego' ,
                 'San Francisco' ,'San Joaquin' ,'San Luis Obispo' ,'San Mateo' ,'Santa Barbara' ,'Santa Clara' ,'Santa Cruz' ,
                 'Shasta' ,'Sierra' ,'Siskiyou' ,'Solano' ,'Sonoma' ,'Stanislaus' ,'Sutter' ,'Tehama' ,'Trinity' ,'Tulare' ,'Tuolumne' ,
                 'Ventura' ,'Yolo' ,'Yuba']

small_county = []
#print(len(counties_list))
for county in counties_list:
    if county.upper() not in cdc_included_counties:
        small_county.append(county)
#print(small_county)

51


In [5]:
#extract only the data for August 2020 on, was originally just for 2021
ca_cases_aug20_on = ca_covid.loc[(ca_covid['case_month'] == '2020-08') |(ca_covid['case_month'] == '2020-09') |
                            (ca_covid['case_month'] == '2020-10') |(ca_covid['case_month'] == '2020-11') |
                             (ca_covid['case_month'] == '2020-12') |
                              (ca_covid['case_month'] == '2021-01') | (ca_covid['case_month'] == '2021-02') | 
                              (ca_covid['case_month'] == '2021-03') | (ca_covid['case_month'] == '2021-04')  
                            | (ca_covid['case_month'] == '2021-05') | (ca_covid['case_month'] == '2021-06') 
                            | (ca_covid['case_month'] == '2021-07') ]                               

#ca_cases_2021 = ca_cases_aug20_on.copy()
#ca_cases_2021 = ca_cases_2021.loc[(ca_cases_2021['case_month'] == '2020-12') |
#                              (ca_cases_2021['case_month'] == '2021-01') | (ca_cases_2021['case_month'] == '2021-02') | 
 #                             (ca_cases_2021['case_month'] == '2021-03') | (ca_cases_2021['case_month'] == '2021-04')  
#                            | (ca_cases_2021['case_month'] == '2021-05') | (ca_cases_2021['case_month'] == '2021-06') 
#                            | (ca_cases_2021['case_month'] == '2021-07') ]  
##print(ca_cases_aug20_on.head())
#print(ca_cases_2021.head())

In [6]:
#write out the modified dataframe
ca_cases_aug20_on.head()
ca_cases_aug20_on.to_csv('./Resources/CA_cases_by_county/ca_aug2020_on_case_data.csv',
                    index=False,header=True)

In [7]:
#convert the case_month to month names so that it can merge with Tikaram
#use DatetimeIndex so that computer does not time out in for loop
ca_cases_aug20_on['month'] = pd.DatetimeIndex(ca_cases_aug20_on['case_month']).month_name()
#ca_cases_2021['month'] = pd.DatetimeIndex(ca_cases_aug20_on['case_month']).month_name()

<ipython-input-7-00d5cdde5380>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca_cases_aug20_on['month'] = pd.DatetimeIndex(ca_cases_aug20_on['case_month']).month_name()


In [8]:
ca_cases_aug20_on.head(3)

,case_month,res_county,age_group,current_status,sex,race,ethnicity,month
0,2020-12,SONOMA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino,December
1,2020-12,SACRAMENTO,18 to 49 years,Laboratory-confirmed case,Male,American Indian/Alaska Native,Hispanic/Latino,December
2,2021-01,SANTA CLARA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino,January


In [9]:
#do this again for August 2020 on
#the warning message is because Jupyter thinks I want the changes in ca_cases_aug20_on
#and ca_cases_2021 linked, though I do not
aug_on_month = ca_cases_aug20_on["month"]

month_order = CategoricalDtype(['August','September','October','November','December',
                                'January', 'February', 'March', 'April',
                                'May', 'June', 'July'], ordered=True)

ca_cases_aug20_on['month'] = ca_cases_aug20_on['month'].astype(month_order)

<ipython-input-9-4181f1df87ae>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca_cases_aug20_on['month'] = ca_cases_aug20_on['month'].astype(month_order)


In [25]:
#this took me a long time to figure out how to do, but it groups by month and county,
#while counting the occurences of each case. .size() makes it into a pandas series
#instead of a groupby object

grouped_by_month = ca_cases_aug20_on.groupby(['month','res_county']).size()
print(grouped_by_month)

#here I convert the pandas series to a df and label the case counts
grouped_by_month_df = grouped_by_month.to_frame(name = 'cases')


month   res_county
August  ALAMEDA       5349
        AMADOR          86
        BUTTE         1252
        CALAVERAS      123
        COLUSA          99
                      ... 
July    TULARE         443
        TUOLUMNE        85
        VENTURA       1202
        YOLO           410
        YUBA           198
Length: 600, dtype: int64


In [29]:
#write the grouped/counted df to csv
grouped_by_month_df.to_csv('./Resources/CA_cases_by_county/cases_by_county_and_month.csv')

In [31]:
#since other code is taking so long, make variables for the case count each month

counts = ca_cases_aug20_on['month'].value_counts()

dec = counts['December']
aug = counts['August']
sept = counts['September']
octo = counts['October']
nov = counts['November']
dec = counts['December']
jan = counts['January']
feb = counts['February']
mar = counts['March']
apr = counts['April']
may = counts['May']
june = counts['June']
july = counts['July']
month_list = counts.tolist()

#print(counts)
#print(type(counts))
#print(month_list)
#print(type(month_list))
ca_cases_aug20_on.head(3)

,case_month,res_county,age_group,current_status,sex,race,ethnicity,month
0,2020-12,SONOMA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino,December
1,2020-12,SACRAMENTO,18 to 49 years,Laboratory-confirmed case,Male,American Indian/Alaska Native,Hispanic/Latino,December
2,2021-01,SANTA CLARA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino,January


In [39]:
total_county_cases_df = ca_cases_aug20_on[['month','res_county']]

#make dataframes of total monthly cases and total cases by county
county_total_cases_df = total_county_cases_df.groupby(by='res_county').size() 
county_total_cases_df = county_counts_df.to_frame(name = 'cases')

total_per_month_df = pd.DataFrame({
    "Months" : ['December','January', 'February', 'March', 'April',
                 'May', 'June', 'July'], 
    "Cases" : [dec,jan,feb,mar,apr,may,june,july]
})

#print(total_per_month_df)
total_per_month_df.to_csv('./Resources/CA_cases_by_county/total_monthly_cases.csv')
county_total_cases_df.to_csv('./Resources/CA_cases_by_county/total_county_cases.csv')

In [ ]:
#get the value counts for demographics and case status, make a list of them,
#and then make a dataframe with all that info

#print(ca_covid.head(70))
age_counts = ca_cases_aug20_on['age_group'].value_counts()
#print(age_counts)
case_status_counts = ca_cases_aug20_on['current_status'].value_counts()
#print(case_status_counts)
sex_counts = ca_cases_aug20_on['sex'].value_counts()
#print(sex_counts)
race_counts = ca_cases_aug20_on['race'].value_counts()
#print(race_counts)
ethnicity_counts = ca_cases_aug20_on['ethnicity'].value_counts()
#print(ethnicity_counts)
county_counts = ca_cases_aug20_on['res_county'].value_counts()


In [ ]:
#show how many cases there are per month
months_strings = ['August','September','October','November','December','January'
                 ,'February', 'March', 'April', 'May','June','July']
x_axis = months_strings
y_axis = month_list

plt.xlabel = ("Month")
plt.ylabel = ('Case Count')
plt.title = ('2021 Cases by Month')


cases_bar_graph = plt.bar(x_axis,y_axis,color = 'b')
graph = plt.gcf()
graph.set_size_inches(14,10)

#plt.savefig('./Resources/CA_cases_by_county/total_cases_by_month.png')




In [ ]:
#months_grouped.reset_index().to_csv('./Resources/CA_cases_by_county/month_grouped_test.csv')

In [ ]:
# make smaller dataframes for each demographic with county, case month, current_status,
# and demographic in the individual dataframes

age_df = ca_covid_2021[['age_group','month', 'res_county', 'current_status']]

#make a smaller dataframe so summing doesn't take as long

age_month_df = pd.DataFrame(age_df, columns=['age_group','month']
age_group_cases_monthly = age_df.groupby(age_df['month']).sum()

In [ ]:
#make the age bar plot
monthly_age_totals_plot = summed_months.plot.bar(color = 'b')
plt.xlabel = ("Month")
plt.ylabel = ('Case Count by Age')
plt.title = ('2021 Age Group Cases by Month')
plt.savefig('./Resources/CA_cases_by_county/total_cases_by_month.png')
plt.show()

In [ ]:
sex_df = ca_covid_2021[['sex','month', 'res_county', 'current_status']]
sex_df.head(3)


In [ ]:
race_df = ca_covid_2021[['race','month', 'res_county', 'current_status']]
race_df.head(3)

In [ ]:
ethnicity_df = ca_covid_2021[['ethnicity','month', 'res_county', 'current_status']]
ethnicity_df.head(3)
ethnicity_df['ethnicity'].value_counts()